In [14]:
import pandas as pd
import numpy as np

In [37]:
class settings(object):
    MYSQL_HOST = '103.110.84.35'
    MYSQL_PORT = 6655
    MYSQL_USER = 'root'
    MYSQL_PASSWORD = 'kvw1d81f'
    MYSQL_DATABASE = 'stockdb'

DATA_PATH = '../data/stock/vn30/'

In [18]:
symbols = ['VN30', 'HPG', 'STB', 'TCB', 'POW', 'SSI', 'CTG', 'MBB', 'VPB', 'VRE',
       'SBT', 'VHM', 'HDB', 'PDR', 'TCH', 'PLX', 'FPT', 'KDH', 'VNM',
       'BID', 'NVL', 'MSN', 'BVH', 'TPB', 'GAS', 'MWG', 'VIC', 'PNJ',
       'VJC', 'VCB', 'REE']

In [19]:
from sqlalchemy import create_engine

class MySQLConnector:
    def __init__(self):
        self.engine = create_engine(f"mysql+mysqlconnector://{settings.MYSQL_USER}:{settings.MYSQL_PASSWORD}@"
                                    f"{settings.MYSQL_HOST}:{settings.MYSQL_PORT}/{settings.MYSQL_DATABASE}")
        self.conn = self.engine.connect()

In [20]:
client = MySQLConnector()

def get_candles(symbols):
    try:
        for symbol in symbols:
            q = f"SELECT t1.timestamp, t1.open, t1.high, t1.low, t1.close, t1.volume FROM stockdb.candles t1 WHERE t1.symbol = '{symbol}' ORDER BY timestamp;"
            df = pd.read_sql(q, client.conn, parse_dates=['timestamp'])
            df.set_index(keys=['timestamp'], inplace=True)
            df.sort_index(inplace=True)
            df.to_csv(f'../data/stock/vn30/{symbol}.csv')
    except Exception as e:
        print(e)
        

In [21]:
get_candles(symbols)

## download VN30 Future data

In [28]:
from datetime import datetime
import requests

VNDIRECT_CHART_URL = 'https://dchart-api.vndirect.com.vn/dchart/'

def get_candles(symbol, start_time: datetime, end_time: datetime, resolution='D'):
    url = f'{VNDIRECT_CHART_URL}history?resolution={resolution}&symbol={symbol}' \
          f'&from={int(round(start_time.timestamp()))}&to={int(round(end_time.timestamp()))}'

    response = requests.request("GET", url).json()
    candles = []
    for t, o, h, l, c, v in zip(response['t'], response['o'], response['h'], response['l'], response['c'],
                                response['v']):
        candles.append({
            'timestamp': datetime.fromtimestamp(int(t)).replace(hour=0),
            'open': o,
            'high': h,
            'low': l,
            'close': c,
            'volume': v,
            'symbol': symbol,
            'resolution': resolution
        })

    return candles

In [29]:
candles_data = get_candles('VN30F1M', start_time=datetime(year=2017, month=8, day=1), end_time=datetime.now())

In [34]:
df = pd.DataFrame(candles_data)
df.set_index(['timestamp'], inplace=True)
df.drop(columns=['symbol', 'resolution'], inplace=True)

In [35]:
df

,open,high,low,close,volume
timestamp,,,,,
2017-11-14,865.5,871.9,863.5,869.7,9399
2017-11-15,871.5,880.2,868.5,877.0,7473
2017-11-16,878.0,888.1,877.0,888.1,6160
2017-11-17,897.5,908.0,891.5,903.0,13311
2017-11-20,904.9,930.0,904.1,930.0,10205
...,...,...,...,...,...
2021-11-08,1537.6,1545.5,1533.6,1537.7,106262
2021-11-09,1538.0,1539.0,1520.5,1525.5,127651
2021-11-10,1525.0,1534.9,1518.6,1529.3,112979


In [36]:
df.to_csv('../data/stock/vn30/VN30F1M.csv')

### Constructing Indexes

In [38]:
market_cap = pd.read_csv(f'{DATA_PATH}VN30_marketcap.csv')
market_cap.head()

,Id,symbol,TradingDate,MarketCapital,Dividend,Yield
0,12488651,BID,2021-11-12 00:00:00,173751.0,NaN,NaN
1,12488511,BVH,2021-11-12 00:00:00,46618.0,NaN,NaN
2,12486487,CTG,2021-11-12 00:00:00,157148.0,NaN,NaN
3,12488315,FPT,2021-11-12 00:00:00,88932.0,NaN,NaN
4,12487229,GAS,2021-11-12 00:00:00,226037.0,NaN,NaN


In [39]:
bank_symbols = ['ACB', 'BID', 'CTG', 'HDB', 'MBB', 'STB', 'TCB', 'TPB', 'VCB']

In [41]:
market_cap[market_cap.symbol.isin(bank_symbols)]

,Id,symbol,TradingDate,MarketCapital,Dividend,Yield
0,12488651,BID,2021-11-12 00:00:00,173751.0,NaN,NaN
2,12486487,CTG,2021-11-12 00:00:00,157148.0,NaN,NaN
5,12486703,HDB,2021-11-12 00:00:00,55782.0,NaN,NaN
8,12489027,MBB,2021-11-12 00:00:00,107871.0,NaN,NaN
19,12487259,STB,2021-11-12 00:00:00,52975.0,NaN,NaN
20,12487495,TCB,2021-11-12 00:00:00,184323.0,NaN,NaN
22,12487777,TPB,2021-11-12 00:00:00,50968.0,NaN,NaN
23,12487813,VCB,2021-11-12 00:00:00,364212.0,NaN,NaN


In [46]:
index = market_cap[market_cap.symbol.isin(bank_symbols)].copy()
index['weight'] = index['MarketCapital'] / index['MarketCapital'].sum()
index.drop(columns=['Id', 'TradingDate', 'Dividend', 'Yield'], inplace=True)
index.to_csv(f'{DATA_PATH}bank_index.csv')

,symbol,MarketCapital,weight
0,BID,173751.0,0.151479
2,CTG,157148.0,0.137004
5,HDB,55782.0,0.048632
8,MBB,107871.0,0.094044
19,STB,52975.0,0.046184
20,TCB,184323.0,0.160696
22,TPB,50968.0,0.044435
23,VCB,364212.0,0.317526


In [50]:
index = market_cap.copy()
index['weight'] = index['MarketCapital'] / index['MarketCapital'].sum()
index.drop(columns=['Id', 'TradingDate', 'Dividend', 'Yield'], inplace=True)
index.to_csv(f'{DATA_PATH}vn30_weights.csv')

In [49]:
index

,symbol,MarketCapital,weight
0,BID,173751.0,0.047069
1,BVH,46618.0,0.012629
2,CTG,157148.0,0.042571
3,FPT,88932.0,0.024092
4,GAS,226037.0,0.061233
5,HDB,55782.0,0.015111
6,HPG,244222.0,0.066159
7,KDH,31825.0,0.008621
8,MBB,107871.0,0.029222
9,MSN,176608.0,0.047843
